In [1]:
#importing libraries
import pandas as pd
import yfinance as yf
from datetime import datetime

In the below code, we first imported the necessary Python libraries and downloaded the historical stock price data for four companies: Apple, Microsoft, Netflix, and Google, for the last three months.

In [28]:
#create the time frame of the data -- 3 months ago from now
start_date = datetime.now() - pd.DateOffset(months=12)
end_date = datetime.now()

In [40]:
#Selecting the company for analysis by ticker symbol
tickers = ['AAPL','MSFT','NFLX','GOOG','AMZN','AMZN']

In [41]:
#create an empty list and append the ticker symbols
df_list =[]

for ticker in tickers:
    data=yf.download(ticker,start=start_date,end=end_date)
    df_list.append(data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [43]:
#concatenate the ticker symbols with the relevant date
df = pd.concat(df_list,keys=tickers,names =['ticker','Date'])

In [44]:
#see the first 10 rows
print(df.head(100))

                         Open        High         Low       Close   Adj Close  \
ticker Date                                                                     
AAPL   2022-06-21  133.419998  137.059998  133.320007  135.869995  135.063507   
       2022-06-22  134.789993  137.759995  133.910004  135.350006  134.546616   
       2022-06-23  136.820007  138.589996  135.630005  138.270004  137.449265   
       2022-06-24  139.899994  141.910004  139.770004  141.660004  140.819138   
       2022-06-27  142.699997  143.490005  140.970001  141.660004  140.819138   
...                       ...         ...         ...         ...         ...   
       2022-11-03  142.059998  142.800003  138.750000  138.880005  138.247406   
       2022-11-04  142.089996  142.669998  134.380005  138.380005  137.978180   
       2022-11-07  137.110001  139.149994  135.669998  138.919998  138.516617   
       2022-11-08  140.410004  141.429993  137.490005  139.500000  139.094940   
       2022-11-09  138.50000

In [45]:
df = df.reset_index()
print(df.head())

  ticker       Date        Open        High         Low       Close  \
0   AAPL 2022-06-21  133.419998  137.059998  133.320007  135.869995   
1   AAPL 2022-06-22  134.789993  137.759995  133.910004  135.350006   
2   AAPL 2022-06-23  136.820007  138.589996  135.630005  138.270004   
3   AAPL 2022-06-24  139.899994  141.910004  139.770004  141.660004   
4   AAPL 2022-06-27  142.699997  143.490005  140.970001  141.660004   

    Adj Close    Volume  
0  135.063507  81000500  
1  134.546616  73409200  
2  137.449265  72433800  
3  140.819138  89116800  
4  140.819138  70207900  


In [46]:
#plotting the chart performance
import plotly.express as px

fig =px.line(df, x ='Date',y='Close',color ='ticker',title="Stock Market Performance for the Last 3 Months")

fig.show()

In [47]:
#comparing perfomance via charts
fig = px.area(df,x = 'Date',y='Close',color ='ticker',facet_col ='ticker',labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},title='Stock Prices for Apple, Microsoft, Netflix, and Google')

fig.show()


In [48]:
#Analysing the moving average as way to analyse trends and pattersn over the 3 months
df['MA10'] = df.groupby('ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
           MA10        MA20
0           NaN         NaN
1           NaN         NaN
2           NaN         NaN
3           NaN         NaN
4           NaN         NaN
..          ...         ...
245  179.752002  176.529002
246  180.353001  177.091001
247  181.023001  177.685001
248  181.615001  178.351000
249  182.012001  178.844500

[250 rows x 2 columns]
Moving Averages for AMZN
            MA10        MA20
1000         NaN         NaN
1001         NaN         NaN
1002         NaN         NaN
1003         NaN         NaN
1004         NaN         NaN
...          ...         ...
1495  123.665001  119.650500
1496  124.165001  120.423501
1497  124.749001  121.074501
1498  125.183001  121.655001
1499  125.307001  122.022001

[500 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
750         NaN         NaN
751         NaN         NaN
752         NaN         NaN
753         NaN         NaN
754         NaN         NaN
..          ...         .

In [49]:
#visualize the moving averages for the stocks
for ticker,group in df.groupby('ticker'):
    fig= px.line(group,x='Date', y=['Close','MA10','MA20'],title = f"{ticker} Moving Averages")

fig.show()

The output shows four separate graphs for each company. When the MA10 crosses above the MA20, it is considered a bullish signal indicating that the stock price will continue to rise. Conversely, when the MA10 crosses below the MA20, it is a bearish signal that the stock price will continue falling.

In [50]:
#Measuring volatility
df['Volatility'] = df.groupby('ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='ticker', 
              title='Volatility of All Companies')
fig.show()

High volatility indicates that the stock or market experiences large and frequent price movements, while low volatility indicates that the market experiences smaller or less frequent price movements.

In [26]:
import statsmodels


In [51]:

#Correlation --AAPL & MSFT
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT', 
                 trendline='ols', 
                 title='Correlation between Apple and Microsoft')
fig.show()

There is a strong linear relationship between the stock prices of Apple and Microsoft, which means that when the stock price of Apple increases, the stock price of Microsoft also tends to increase. It is a sign of a strong correlation or similarity between the two companies, which can be due to factors such as industry trends, market conditions, or common business partners or customers. For investors, this positive correlation may indicate an opportunity to diversify their portfolio by investing in both companies, as both stocks may offer similar potential returns and risks.

So this is how you can perform stock market analysis using Python.